# Calculate height above DEM

<b>This Jupyter-Notebook is part of a master thesis with the topic:<br>
    <i>Analysis of deep learning methods for semantic segmentation of photogrammetric point clouds from aerial images</i><br>
&copy; Markus Hülsen, Matr.-Nr. 6026370<br>
Date: 27.06.2023</b>

In our Notebook `LAS_to_DEM` we generated the DEM from our ALS pointclod.<br>
Now we use these elevation models to calculate the height above DEM. <br>
We add this feature as a scalarfield to the point cloud an we will save and export the pointcloud as las-file.

In [1]:
import os
import laspy
import pandas as pd
import numpy as np

## Data import
First we need to define the import path of our DEM and our pointcloud.

In [2]:
# path where our data is stored
data_path = '../../Daten/Datensatz_H3D/'
las_path = data_path + 'DIM_2022/1 - after_CSF'

Now get the paths of the las-files inside the seleceted folder.

In [3]:
# save files that are in laz-format
lst_files = []
for file in os.listdir(las_path):
    if file.endswith('.las'):
        lst_files.append(las_path + '/' + file)

# sort list
lst_files = sorted(lst_files)
print('Found', len(lst_files), 'las-files:')
print(lst_files)

Found 12 las-files:
['../../Daten/Datensatz_H3D/DIM_2022/1 - after_CSF/554000_5798000.las', '../../Daten/Datensatz_H3D/DIM_2022/1 - after_CSF/554000_5799000.las', '../../Daten/Datensatz_H3D/DIM_2022/1 - after_CSF/554000_5800000.las', '../../Daten/Datensatz_H3D/DIM_2022/1 - after_CSF/554000_5801000.las', '../../Daten/Datensatz_H3D/DIM_2022/1 - after_CSF/555000_5798000.las', '../../Daten/Datensatz_H3D/DIM_2022/1 - after_CSF/555000_5799000.las', '../../Daten/Datensatz_H3D/DIM_2022/1 - after_CSF/555000_5800000.las', '../../Daten/Datensatz_H3D/DIM_2022/1 - after_CSF/555000_5801000.las', '../../Daten/Datensatz_H3D/DIM_2022/1 - after_CSF/556000_5798000.las', '../../Daten/Datensatz_H3D/DIM_2022/1 - after_CSF/556000_5799000.las', '../../Daten/Datensatz_H3D/DIM_2022/1 - after_CSF/556000_5800000.las', '../../Daten/Datensatz_H3D/DIM_2022/1 - after_CSF/556000_5801000.las']


Define which file we will use.

In [4]:
las_file = lst_files[0]
las_file

'../../Daten/Datensatz_H3D/DIM_2022/1 - after_CSF/554000_5798000.las'

### Import of the point cloud
Function to import a pointcloud as DataFrame

In [5]:
def import_las_to_Dataframe(path):
    with laspy.open(path) as f:
        las = f.read()
    
    # read coordinates from las
    x = np.array(las.x)
    y = np.array(las.y)
    z = np.array(las.z)

    df = pd.DataFrame({'X':x,'Y':y,'Z':z},index=np.arange(len(x)))

    for i in range(3, len(las.point_format.dimensions)):
        dim = las.point_format.dimensions[i].name
        df[dim] = np.array(las[dim])
     
    return df

Now import the pointcloud

In [6]:
df_pc = import_las_to_Dataframe(las_file)
df_pc = df_pc.drop('Original__cloud__index', axis=1, errors='ignore')
df_pc

,X,Y,Z,intensity,return_number,number_of_returns,scan_direction_flag,edge_of_flight_line,classification,synthetic,key_point,withheld,scan_angle_rank,user_data,point_source_id,gps_time,red,green,blue,delta_z
0,554024.09,5798002.83,66.05,10442,1,7,0,0,2,0,0,0,-110,116,0,0.0,55040,54016,52224,0.188888
1,554026.47,5798000.37,65.92,16696,1,2,0,0,2,0,0,0,126,200,0,0.0,42240,33280,27904,0.087113
2,554026.48,5798005.19,65.71,9527,1,2,0,0,2,0,0,0,123,123,0,0.0,9728,15616,19200,0.559579
3,554026.66,5798005.76,65.80,9527,1,6,0,0,2,0,0,0,-116,113,0,0.0,6656,13056,15616,0.989986
4,554049.63,5798046.24,64.24,10094,1,1,0,0,2,0,0,0,113,40,0,0.0,38144,36352,32000,-0.378887
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5631159,554999.26,5798835.98,84.83,7364,1,3,0,0,13,0,0,0,-110,138,0,0.0,16384,27392,20992,11.483861
5631160,554989.94,5798860.83,83.46,7366,1,2,0,0,13,0,0,0,85,102,0,0.0,26880,38400,26112,9.923417
5631161,554999.75,5798837.19,84.38,7366,1,2,0,0,13,0,0,0,-79,135,0,0.0,8960,15872,13312,10.909648
5631162,554994.31,5798832.62,85.54,7361,1,3,0,0,13,0,0,0,104,97,0,0.0,14336,23296,18944,14.007186


### Import of the DEM
Define the DEM path

In [7]:
dem_path = data_path + 'DEM/' + las_file.split('/')[-1].replace('.las','') + '.csv'
dem_path

'../../Daten/Datensatz_H3D/DEM/554000_5798000.csv'

Import the DEM as Pandas DataFrame.

In [8]:
df_dem = pd.read_csv(dem_path, sep='\t').set_index('Unnamed: 0')
df_dem

,X,Y,Z
Unnamed: 0,,,
0,554000.000000,5798000.000,64.262001
1,554001.000969,5798000.000,64.278496
2,554002.001938,5798000.000,64.319000
3,554003.002907,5798000.000,64.329750
4,554004.003876,5798000.000,64.370598
...,...,...,...
999995,554995.964124,5798999.749,68.209814
999996,554996.965093,5798999.749,68.209814
999997,554997.966062,5798999.749,68.209814


## Delaunay Triangulation
For interpolation we use the Delaunay Triangulation, which is included in the `scipy`-Library.

In [9]:
from scipy.spatial import Delaunay

Here we will generate a Delaunay Triangulation for the 2D poinnt cloud - so we will just use the X- and Y-coordinates.

In [10]:
# Create a Delaunay triangulation object from the input points.
tri = Delaunay(df_dem.loc[:,'X':'Y'].to_numpy())

### Interpolate with `LinearNDInterpolator`
To calculate the distance from our Triangulation to our points, we use the `LinearNDInterpolator`.

In [11]:
from scipy.interpolate import LinearNDInterpolator

First we will initialize the interpolator and add the Z-coordinates as values to our triangulation.

In [12]:
# initalize the interpolator with the triangulation and the z-coordinate of the DEM
interp = LinearNDInterpolator(tri, df_dem.Z.to_numpy())

Now we can use the interpolator to get the DEM height for every point.

In [13]:
# Use interpolator to interpolate the z-coords of our points inside the Triangulation
z_dem = interp(df_pc.loc[:,'X':'Y'].to_numpy())
z_dem

array([65.96411172, 65.93588682, 65.25342119, ..., 73.57435189,
       71.63681449, 70.86931673])

Calculate the difference between the Z-coordinate of the points and the DEM-height.

In [14]:
delta_z = df_pc.Z.to_numpy() - z_dem
delta_z

array([ 0.08588828, -0.01588682,  0.45657881, ..., 10.80564811,
       13.90318551, 15.07068327])

## Add results to the Point Cloud
Now that we calculated the height above DEM we can add the results to the DataFrame

In [15]:
df_pc['z_to_dem'] = delta_z
df_pc

,X,Y,Z,intensity,return_number,number_of_returns,scan_direction_flag,edge_of_flight_line,classification,synthetic,...,withheld,scan_angle_rank,user_data,point_source_id,gps_time,red,green,blue,delta_z,z_to_dem
0,554024.09,5798002.83,66.05,10442,1,7,0,0,2,0,...,0,-110,116,0,0.0,55040,54016,52224,0.188888,0.085888
1,554026.47,5798000.37,65.92,16696,1,2,0,0,2,0,...,0,126,200,0,0.0,42240,33280,27904,0.087113,-0.015887
2,554026.48,5798005.19,65.71,9527,1,2,0,0,2,0,...,0,123,123,0,0.0,9728,15616,19200,0.559579,0.456579
3,554026.66,5798005.76,65.80,9527,1,6,0,0,2,0,...,0,-116,113,0,0.0,6656,13056,15616,0.989986,0.886986
4,554049.63,5798046.24,64.24,10094,1,1,0,0,2,0,...,0,113,40,0,0.0,38144,36352,32000,-0.378887,-0.481887
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5631159,554999.26,5798835.98,84.83,7364,1,3,0,0,13,0,...,0,-110,138,0,0.0,16384,27392,20992,11.483861,11.379861
5631160,554989.94,5798860.83,83.46,7366,1,2,0,0,13,0,...,0,85,102,0,0.0,26880,38400,26112,9.923417,9.819417
5631161,554999.75,5798837.19,84.38,7366,1,2,0,0,13,0,...,0,-79,135,0,0.0,8960,15872,13312,10.909648,10.805648
5631162,554994.31,5798832.62,85.54,7361,1,3,0,0,13,0,...,0,104,97,0,0.0,14336,23296,18944,14.007186,13.903186


## Export DataFrame
Now we have everything we wanted to calculate. Next we will export the pointcloud <br>
First we define a function to export a DataFrame to an las-file.

In [16]:
def save_df_to_las(df, path):
    
    header = laspy.LasHeader(point_format=3, version="1.2")
    
    atts = []
    for dim in header.point_format.dimensions:
        atts.append(dim.name)
    
    for dim in df.columns:
        if dim not in atts:
            header.add_extra_dim(laspy.ExtraBytesParams(name=dim, type=np.float32))
    
    las_new = laspy.LasData(header)

    las_new.x = df.X.to_numpy()
    las_new.y = df.Y.to_numpy()
    las_new.z = df.Z.to_numpy()
    
    for col in df.loc[:,'intensity':].columns:
        las_new[col] = df[col].to_numpy()
    
    las_new.write(path)
    print('LAS file has been exported to ', path)

Define export path.

In [17]:
export_path = las_file.replace('1 - after_CSF','2 - Height above DEM')

Save the DataFrame to the defined path.

In [18]:
save_df_to_las(df_pc, export_path)

LAS file has been exported to  ../../Daten/Datensatz_H3D/DIM_2022/2 - Height above DEM/554000_5798000.las
